# Dash Server

## Setup

In [1]:
import numpy as np
import pandas as pd

### Logging

In [2]:
from myutils import customlogging
import importlib
importlib.reload(customlogging)
mylogger = customlogging.create_dual_logger(__name__, 'log/dashlog.log', file_reset=True)
mylogger

06-Jul-20 11:05:37 - __main__ - INFO     - Logger __main__ starting...


<Logger __main__ (DEBUG)>

### Libraries

In [3]:
from myutils import generic, betting, timing, guiserver, importer
importer.reload_utils()

20-07-06 11-05-38 - utils lib reloaded


### Historical List

In [4]:
trading = betting.get_api_client()
trading.login()

historical_queue = betting.get_historical(trading, r'data/bfsamplerace')
historical_list = list(historical_queue.queue)
print(f'Data entries: {len(historical_list)}')

Data entries: 20730


In [56]:
t = historical_list[1000][0].publish_time
@timing.decorator_timer
def doit():
    i = 0
    while i < len(historical_list) and historical_list[i][0].publish_time < t:
        i += 1
doit()

Finished doit in 0.0051 seconds


In [75]:
prerace = [h for h in historical_list if not h[0].inplay]
lastRecord = prerace[-1][0]

[[r.name, r.sort_priority] for r in lastRecord.market_definition.runners[:3]]

[['Golden V C', 1], ['Bahamian Star', 2], ['Impressive Speed', 3]]

In [81]:
names = betting.get_names(lastRecord.market_definition)

In [92]:
def sorter(runner):
    ltp = runner.last_price_traded
    if ltp:
        return ltp
    else:
        return float('inf')

In [93]:
[[names[r.selection_id], r.last_price_traded] for r in sorted(lastRecord.runners, key=sorter)]

[['Federale', 1.74],
 ['Large', 8.6],
 ['Burl', 15.0],
 ['Puerto Varas', 17.5],
 ['Impressive Speed', 19.0],
 ['Bahamian Star', 26.0],
 ['Robertos Ticket', 27.0],
 ['Adios See Ya', 36.0],
 ['Golden V C', 46.0]]

In [ ]:
[r]

## Indexing book against ticks

Could use numpy or regular lists, but should check speed first

In [5]:
from myutils.timing import decorator_timer
import numpy as np

l = range(100000)
npl = np.array(l)

@decorator_timer
def nptestin(x):
    w = np.where(npl==x)[0]
    if w.shape[0]:
        return w[0]
    else:
        print('not found')
        
@decorator_timer
def testin(x):
    if x in l:
        return l.index(x)
    else:
        print('not found')

In [6]:
testin(999999)

not found
Finished testin in 0.0001 seconds


In [7]:
nptestin(999999)

not found
Finished nptestin in 0.0006 seconds


# Dash

Code for dash gui is stored in `gui.py`  
Need to re-import each time running incase of changes

In [98]:
import gui
import importlib
importlib.reload(gui)

08-Jul-20 11:53:04 - gui - INFO     - Logger gui starting...


<module 'gui' from 'C:\\Users\\joele\\Documents\\coding_projects\\betfairgui_dash\\gui.py'>

In [99]:
gui.run(__name__, historical_list, False)

08-Jul-20 11:53:06 - __main__ - INFO     - Dash is running on http://127.0.0.1:8050/



Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/



08-Jul-20 11:53:06 - __main__ - INFO     -  Warning: This is a development server. Do not use app.run_server


08-Jul-20 11:53:06 - __main__ - INFO     -  in production, use a production WSGI server like gunicorn instead.



 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Jul/2020 11:53:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2020 11:53:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2020 11:53:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2020 11:53:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2020 11:53:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2020 11:53:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2020 11:53:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2020 11:53:10] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2020 11:53:10] "GET /_dash-layout HTTP/1.1" 200 -
08-Jul-20 11:53:10 - gui - WARNING  - Button pressed not recognised: .
08-Jul-20 11:53:10 - gui - INFO     - slider id "." not recognised
127.0.0.1 - - [08/Jul/2020 11:53:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 -